In [3]:
# GENERAR PDF CON LA COMPARATIVA DE IMÁGENES 1:1 ENTRE MODELO MULTIMODAL Y "GROUND TRUTH"
from pathlib import Path
import pandas as pd


path = Path("../../data/flickr/flickr_validated_imgs_7000")
categories = pd.read_csv("../../data/categories/categories_stoten.csv", sep=";", header=None)
llava_classification = pd.read_csv("../../data/inference/stoten_w_descriptions_plus_not_relevant.csv", sep=";", header=0)
manual_classification = pd.read_csv("../../data/inference/stoten_manual_annotation.csv", sep=";", header=0)


image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
# Find all image files recursively and filter by extension (lowercase only)
image_paths = [img_path for img_path in path.rglob('*') if img_path.suffix.lower() in image_extensions]
# Convert to lowercase and remove duplicates (especially relevant for Windows)
unique_image_paths = {img_path.resolve().as_posix().lower(): img_path for img_path in image_paths}
images = list(unique_image_paths.values())


# Categorías a analizar
categories_list = categories.iloc[:, 0].astype(str).str.upper().tolist()


# Cogemos inferencia de llava
llava_classification["img"] = llava_classification["img"].apply(lambda x: x.split("/")[-1])
llava_classification = llava_classification[["img","category_llava"]]
llava_classification['category_llava'] = llava_classification['category_llava'].apply(lambda x: x.upper())
# Las categorías que no están en la lista de categorías, han sido malas inferencias. 
# 1. Hay varias categorías que se pueden afinar. Por ejemplo, el modelo ha puesto SPIRITUAL, y no la frase completa
# de Spiritual, symbolic and related connotations
llava_classification["category_llava"] = llava_classification["category_llava"].replace("SPIRITUAL","SPIRITUAL, SYMBOLIC AND RELATED CONNOTATIONS") 
llava_classification["category_llava"] = llava_classification["category_llava"].replace("SYMBOLIC AND RELATED CONNOTATIONS","SPIRITUAL, SYMBOLIC AND RELATED CONNOTATIONS") 
llava_classification["category_llava"] = llava_classification["category_llava"].apply(
                    lambda cat: cat if cat in categories_list or cat == "NOT VALID" or cat == "NOT RELEVANT" else "BAD_INFERENCE"
                    )


# Mergeamos con el resultado de etiquetado manual
# ATENCIÓN, HAY NAs NO CLASIFICADO EN STOTEN. Ponemos Other type
manual_classification['manual_category'] = manual_classification['manual_category'].fillna("Other type")
manual_classification['manual_category'] = manual_classification['manual_category'].apply(lambda x: x.upper())
# Unimos ambos por imagen
lvlm_manual = llava_classification.merge(manual_classification[["img","manual_category"]],on="img",how="left")


# create_pdf_from_images("images.pdf",images, result)


In [4]:
import os
from math import ceil
from pathlib import Path

import pandas as pd
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image, PageBreak, Flowable
)
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors

# -------------------------------
# Flowable personalizado
# -------------------------------
class ImageAndCaption(Flowable):
    """
    Flowable que dibuja una imagen y, debajo, un párrafo (caption).
    """
    def __init__(self, image_flowable, caption_paragraph):
        super().__init__()
        self.image_flowable = image_flowable
        self.caption_paragraph = caption_paragraph
        # Anchura/altura calculada en wrap()
        self.width = 0
        self.height = 0

    def wrap(self, availWidth, availHeight):
        """
        Determina cuánto espacio vertical (y horizontal) necesita
        este flowable para que quepa imagen + caption.
        """
        i_w, i_h = self.image_flowable.wrap(availWidth, availHeight)
        c_w, c_h = self.caption_paragraph.wrap(availWidth, availHeight)

        self.width = max(i_w, c_w)
        self.height = i_h + c_h
        return (self.width, self.height)

    def draw(self):
        """
        Dibuja primero la imagen y después el caption debajo.
        """
        # Empaquetar para saber su altura efectiva
        i_w, i_h = self.image_flowable.wrap(self.width, self.height)
        # Dibuja la imagen en la parte superior
        self.image_flowable.drawOn(self.canv, 0, self.height - i_h)

        c_w, c_h = self.caption_paragraph.wrap(self.width, self.height)
        # Dibuja el caption debajo de la imagen
        self.caption_paragraph.drawOn(self.canv, 0, self.height - i_h - c_h)

# -------------------------------
# Generar PDF paginado (8x5)
# -------------------------------
def create_pdf_from_images(path_save_pdf, images, lvlm_manual):
    # Parámetros de tabla
    filas = 6
    columnas = 5
    max_imgs = filas * columnas  # 40 por página

    # Crear el documento
    doc = SimpleDocTemplate(path_save_pdf, pagesize=letter)
    story = []
    estilos = getSampleStyleSheet()
    estilo_normal = estilos['Normal']
    estilo_normal.leading = 5       # Altura de línea
    estilo_normal.fontSize = 4  # ← línea a modificar para letra más pequeña
    estilo_rojo = estilos['Normal'].clone('rojo')
    estilo_rojo.textColor = colors.red


    num_paginas = ceil(len(images) / max_imgs)

    for p in range(num_paginas):
        # Subconjunto de imágenes para esta página
        subset_imagenes = images[p * max_imgs : (p + 1) * max_imgs]

        tabla_datos = []
        fila_actual = []

        for ruta in subset_imagenes:
            # Crear el flowable de la imagen con tamaño fijo
            img = Image(str(ruta), width=80, height=80)

            # Obtener nombre para buscar en DataFrame
            nombre_imagen = str(Path(ruta).resolve()).split("/")[-1]
            fila_df = lvlm_manual[lvlm_manual['img'] == nombre_imagen]

            # Crear el párrafo
            if not fila_df.empty:
                category_llava = fila_df.iloc[0]['category_llava']
                manual_category = fila_df.iloc[0]['manual_category']
                if category_llava != manual_category:
                    caption = Paragraph(f"{nombre_imagen.split('.')[0]}<br/>{category_llava} <br/> {manual_category}", estilo_rojo)
                else:
                    caption = Paragraph(f"{nombre_imagen.split('.')[0]}<br/>{category_llava} <br/> {manual_category}", estilo_normal)
            else:
                caption = Paragraph("N/A", estilo_normal)

            # Combinar imagen + texto en un flowable
            celda = ImageAndCaption(img, caption)
            fila_actual.append(celda)

            # Si se llenó la fila (5 columnas), la agregamos a la tabla
            if len(fila_actual) == columnas:
                tabla_datos.append(fila_actual)
                fila_actual = []

        # Si la última fila quedó incompleta, la rellenamos
        if fila_actual:
            while len(fila_actual) < columnas:
                fila_actual.append(Paragraph(" ", estilo_normal))
            tabla_datos.append(fila_actual)

        # Rellenar filas restantes para asegurar 8 filas
        while len(tabla_datos) < filas:
            tabla_datos.append([Paragraph(" ", estilo_normal)] * columnas)

        # Construir la tabla de 8x5
        tabla = Table(
            tabla_datos,
            colWidths=[90]*columnas,
            rowHeights=[100]*filas
        )
        tabla.setStyle(TableStyle([
            ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ]))

        story.append(tabla)
        story.append(Spacer(1, 12))

        # Si no es la última página, forzamos salto
        if p < num_paginas - 1:
            story.append(PageBreak())

    # Construimos el PDF
    doc.build(story)

create_pdf_from_images("images.pdf", images, lvlm_manual)
